Posted a lot of Charles Darwin's books below. His books are analyzed. Dendrograms are created.

In [1]:
# Import library
import glob

# The books files are contained in this folder
folder = "books/"

# List all the .txt files and sort them alphabetically
files = glob.glob(folder + "*.txt")
files.sort()
files

['books/Autobiography.txt',
 'books/CoralReefs.txt',
 'books/DescentofMan.txt',
 'books/DifferentFormsofFlowers.txt',
 'books/EffectsCrossSelfFertilization.txt',
 'books/ExpressionofEmotionManAnimals.txt',
 'books/FormationVegetableMould.txt',
 'books/FoundationsOriginofSpecies.txt',
 'books/GeologicalObservationsSouthAmerica.txt',
 'books/InsectivorousPlants.txt',
 'books/LifeandLettersVol1.txt',
 'books/LifeandLettersVol2.txt',
 'books/MonographCirripedia.txt',
 'books/MonographCirripediaVol2.txt',
 'books/MovementClimbingPlants.txt',
 'books/OriginofSpecies.txt',
 'books/PowerMovementPlants.txt',
 'books/VariationPlantsAnimalsDomestication.txt',
 'books/VolcanicIslands.txt',
 'books/VoyageBeagle.txt']

In [2]:
# Import libraries
import re, os

# Initialize the object that will contain the texts and titles
txts = []
titles = []

for n in files:
    # Open each file
    f = open(n, encoding='utf-8-sig')
    # Remove all non-alpha-numeric characters
    data = re.sub('[\W_]+', ' ', f.read())
    # Store the texts and titles of the books in two separate lists
    txts.append(data)
    titles.append(os.path.basename(n).replace(".txt", ""))
    
# Print the length, in characters, of each book
[len(t) for t in txts]

[123231,
 496068,
 1776539,
 617088,
 913713,
 624232,
 335920,
 523021,
 797401,
 901406,
 1047518,
 1010643,
 767492,
 1660866,
 298319,
 916267,
 1093567,
 1043499,
 341447,
 1149574]

In [3]:
print(titles)

['Autobiography', 'CoralReefs', 'DescentofMan', 'DifferentFormsofFlowers', 'EffectsCrossSelfFertilization', 'ExpressionofEmotionManAnimals', 'FormationVegetableMould', 'FoundationsOriginofSpecies', 'GeologicalObservationsSouthAmerica', 'InsectivorousPlants', 'LifeandLettersVol1', 'LifeandLettersVol2', 'MonographCirripedia', 'MonographCirripediaVol2', 'MovementClimbingPlants', 'OriginofSpecies', 'PowerMovementPlants', 'VariationPlantsAnimalsDomestication', 'VolcanicIslands', 'VoyageBeagle']


In [4]:
# Browse the list containing all the titles
for i in range(len(titles)):
    # Store the index if the title is "OriginofSpecies"
    if(titles[i]=="OriginofSpecies"):
        ori = i

        
# Print the stored index
print(str(ori))

15


In [5]:
# Define a list of stop words
stoplist = set('for a of the and to in to be which some is at that we i who whom show via may my our might as well'.split())

# Convert the text to lower case 
txts_lower_case = [txt.lower() for txt in txts]

# Transform the text into tokens 
txts_split = [txt.split() for txt in txts_lower_case]

# Remove tokens which are part of the list of stop words
texts = [[word for word in txt if word not in stoplist] for txt in txts_split]

# Print the first 20 tokens for the "On the Origin of Species" book
texts[ori][0:20]

['on',
 'origin',
 'species',
 'but',
 'with',
 'regard',
 'material',
 'world',
 'can',
 'least',
 'go',
 'so',
 'far',
 'this',
 'can',
 'perceive',
 'events',
 'are',
 'brought',
 'about']

In [6]:
# Load the Porter stemming function from the nltk package
from nltk.stem import PorterStemmer

# Create an instance of a PorterStemmer object
porter = PorterStemmer()

# For each token of each text, we generated its stem 
texts_stem = [[porter.stem(token) for token in text] for text in texts]

# Save to pickle file
pickle.dump( texts_stem, open( "books/texts_stem2.p", "wb" ) )

NameError: name 'pickle' is not defined

In [ ]:
import pickle

# Load the stemmed tokens list from the pregenerated pickle file
texts_stem = pickle.load(open("books/texts_stem2.p", "rb"))

# Print the 20 first stemmed tokens from the "On the Origin of Species" book
print(texts_stem[ori][0:20])

In [ ]:
# Load the functions allowing to create and use dictionaries
from gensim import corpora

# Create a dictionary from the stemmed tokens
dictionary = corpora.Dictionary(texts_stem)

# Create a bag-of-words model for each book, using the previously generated dictionary
bows = [dictionary.doc2bow(text) for text in texts_stem]


# Print the first five elements of the On the Origin of species' BoW model
bows[ori][0:5]

In [ ]:
# Import pandas to create and manipulate DataFrames
import pandas as pd

# Convert the BoW model for "On the Origin of Species" into a DataFrame
df_bow_origin = pd.DataFrame(bows[ori])

# Add the column names to the DataFrame
df_bow_origin.columns = ["index", "occurrences"]

# Add a column containing the token corresponding to the dictionary index
df_bow_origin["token"] = [dictionary[index] for index in df_bow_origin["index"]]

# Sort the DataFrame by descending number of occurrences and print the first 10 values
df_bow_origin.sort_values(by="occurrences", ascending=False).head(10)

In [ ]:
# Load the gensim functions that will allow us to generate tf-idf models
from gensim.models import TfidfModel

# Generate the tf-idf model
model = TfidfModel(bows)

# Print the model for "On the Origin of Species"
model[bows[ori]]

In [ ]:
# Convert the tf-idf model for "On the Origin of Species" into a DataFrame
df_tfidf = pd.DataFrame(model[bows[ori]])

# Name the columns of the DataFrame id and score
df_tfidf.columns=["id", "score"]

# Add the tokens corresponding to the numerical indices for better readability
df_tfidf['token'] = [dictionary[i] for i in list(df_tfidf["id"])]

# Sort the DataFrame by descending tf-idf score and print the first 10 rows.
df_tfidf.sort_values(by="score", ascending=False).head(10)

In [ ]:
# Load the library allowing similarity computations
from gensim import similarities

# Compute the similarity matrix (pairwise distance between all texts)
sims = similarities.MatrixSimilarity(model[bows])

# Transform the resulting list into a DataFrame
sim_df = pd.DataFrame(list(sims))

# Add the titles of the books as columns and index of the DataFrame
sim_df.columns = titles
sim_df.index = titles

# Print the resulting matrix
sim_df

In [ ]:
# This is needed to display plots in a notebook
%matplotlib inline

# Import the needed functions from matplotlib
import matplotlib.pyplot as plt

# Select the column corresponding to "On the Origin of Species" and 
v = sim_df["OriginofSpecies"]

# Sort by ascending scores
v_sorted = v.sort_values(ascending=True)

# Plot this data has a horizontal bar plot
v_sorted.plot.barh(x='lab', y='val', rot=0).plot()

# Modify the axes labels and plot title for better readability
plt.xlabel("Cosine distance")
plt.ylabel("")
plt.title("Most similar books to 'On the Origin of Species'")

In [ ]:
# Import libraries
from scipy.cluster import hierarchy

# Compute the clusters from the similarity matrix,
# using the Ward variance minimization algorithm
Z = hierarchy.linkage(sim_df, 'ward')

# Display this result as a horizontal dendrogram
a = hierarchy.dendrogram(Z,  leaf_font_size=8, labels=sim_df.index,  orientation="left")